<a href="https://colab.research.google.com/github/SaiidAmiri/AIS_Aufgabenstellung/blob/main/Said_Amiri_Aufgabenstellung_AIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0- Bibliotheken und Datensätze importieren**

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# Datensätze importieren
Kundendatei_path = '/gdrive/MyDrive/AIS_Management/Kundendatei.xlsx'
Beispielobjekte_path = '/gdrive/MyDrive/AIS_Management/Beispielobjekte.xlsx'
EP_Katalog_path = '/gdrive/MyDrive/AIS_Management/EP_Katalog.xlsx'
df_Kunden = pd.read_excel(Kundendatei_path)
df_Beispielobjekt = pd.read_excel(Beispielobjekte_path)
df_EP_Katalog = pd.read_excel(EP_Katalog_path)

**1. Vollständigkeitsprüfung**

In [4]:
def Vollständigkeit(df_Kunden, df_Beispielobjekt, model):
  """
    Prüft, ob typische Anlagen pro Gebäude aus den Beispielobjekte im Kundendatensatz enthalten sind.

    Vergleicht Kundenanlagen mit typischen Referenzanlagen anhand semantischer Ähnlichkeit
    (SentenceTransformer-Modell).

    Parameter:
        df_Kunden (pd.DataFrame): Kundendaten mit 'WirtEinh' und 'EQ-Klasse-Bezeichnung'.
        df_Beispielobjekt (pd.DataFrame): Referenzanlagen mit 'AKS-Bezeichnung'.
        model: Ein SentenceTransformer-Modell zur Vektor-Repräsentation der Anlagentexte.

    Rückgabe:
        pd.DataFrame: Tabelle mit Gebäude, Anlage, höchste berechnete Ähnlichkeit zur Kundenanlage und Status ('✅' oder '❌').
  """
  # Liste typischer Anlagen
  typische_anlagen = df_Beispielobjekt["AKS-Bezeichnung"].dropna().drop_duplicates()
  # Alle Gebäude
  gebaeude_ids = df_Kunden["WirtEinh"].drop_duplicates()
  # Lade und bereite Daten vor
  result = []

  # Pre-encode Referenzanlagen für Effizienz
  typische_anlagen_list = [str(a) for a in typische_anlagen]
  typische_embeddings = model.encode(typische_anlagen_list, convert_to_tensor=True)
  for gid in gebaeude_ids:
    # Kundenanlagen pro Gebäude
    gebaeude_anlagen = df_Kunden[df_Kunden["WirtEinh"] == gid]["EQ-Klasse-Bezeichnung"].dropna().unique()
    gebaeude_anlagen_list = [str(a) for a in gebaeude_anlagen]
    kunden_embeddings = model.encode(gebaeude_anlagen_list, convert_to_tensor=True)

    # Ähnlichkeitsmatrix: Kundenanlagen vs. Referenzanlagen
    cosine_similarities = util.cos_sim(kunden_embeddings, typische_embeddings)

    # max pro Spalte (jede Referenzanlage → höchster Match mit einer Kundenanlage)
    for idx, referenz in enumerate(typische_anlagen_list):
        max_similarity = cosine_similarities[:, idx].max().item()
        vorhanden = max_similarity > 0.7  # Schwellenwert (anpassbar)
        result.append({
            "Gebäude": gid,
            "Typische Anlage": referenz,
            "Maximale Ähnlichkeit": round(max_similarity, 2),
            "Status": "✅ Vorhanden" if vorhanden else "❌ Fehlend"
        })

  return pd.DataFrame(result)

In [5]:
# SentenceTransformer-Modell zur Vektor-Repräsentation der Anlagentexte
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Ergebnisdatei zur Vollständigkeitsprüfung
df_Vergleich = Vollständigkeit(df_Kunden, df_Beispielobjekt, model)
df_Vergleich

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,Gebäude,Typische Anlage,Maximale Ähnlichkeit,Status
0,21083,Kompressionskältemaschine,0.58,❌ Fehlend
1,21083,Verteilnetz / Pumpen - Kälteerzeugung,0.61,❌ Fehlend
2,21083,Heizkörper,0.64,❌ Fehlend
3,21083,Wärmeverteilnetz,0.57,❌ Fehlend
4,21083,Begleitheizung,0.55,❌ Fehlend
...,...,...,...,...
786,26287,Uhrenanlage,0.56,❌ Fehlend
787,26287,Falt- und Schiebewand,0.54,❌ Fehlend
788,26287,Abwassersammelanlage,0.82,✅ Vorhanden
789,26287,"Leitern, Tritte ortsveränderl.",0.36,❌ Fehlend


In [7]:
# Excel-Datei exportieren
file_path = '/gdrive/MyDrive/AIS_Management/Vollständigkeitsprüfung.xlsx'
df_Vergleich.to_excel(file_path, index=False)

**2. EP-Zuordnung**

In [6]:
def EP_Zuordnung(df_Kunden, df_EP_Katalog):
  """
    Ordnet jeder Kundenanlage die passendste Artikelnummer aus dem EP-Katalog zu.

    Nutzt TF-IDF-Vektorisierung und Cosine Similarity, um Kundenbeschreibungen mit
    Katalogtexten zu vergleichen.

    Parameter:
        df_Kunden (pd.DataFrame): Kundendaten mit Anlagenbezeichnungen.
        df_EP_Katalog (pd.DataFrame): EP-Katalog mit 'Artikelnummer' und 'Kurztext / Bezeichnung'.

    Rückgabe:
        pd.DataFrame: Erweiterter Kundendatensatz mit zugeordneter Artikelnummer und Kurztext.
  """
  # Nur Zeilen mit Artikelnummern
  df_EP_Katalog = df_EP_Katalog[df_EP_Katalog["Artikelnummer"].notna()]

  # Texte zusammenstellen
  kunden_texts = df_Kunden["EQ-Bezeichnung"].fillna('') + " " + \
               df_Kunden["EQ-Klasse-Bezeichnung"].fillna('') + " " + \
               df_Kunden["Anlagenausprägung"].fillna('')

  ep_texts = df_EP_Katalog["Kurztext / Bezeichnung"].fillna('')

  # TF-IDF-Vektoren berechnen
  vectorizer = TfidfVectorizer().fit(pd.concat([kunden_texts, ep_texts]))
  kunden_vecs = vectorizer.transform(kunden_texts)
  ep_vecs = vectorizer.transform(ep_texts)

  # Ähnlichkeiten berechnen
  similarity_matrix = cosine_similarity(kunden_vecs, ep_vecs)
  best_matches = similarity_matrix.argmax(axis=1)

  # Zuordnungen einfügen
  df_Kunden["Zugeordnete Artikelnummer"] = df_EP_Katalog.iloc[best_matches]["Artikelnummer"].values
  df_Kunden["Artikel Kurztext"] = df_EP_Katalog.iloc[best_matches]["Kurztext / Bezeichnung"].values

  return df_Kunden

In [8]:
# Ergebnisdatei zur EP-Zuordnung
df_EP_Zuordnung = EP_Zuordnung(df_Kunden, df_EP_Katalog)
df_EP_Zuordnung

,WirtEinh,EQ_übergeordnet,Equipment,EQ-Bezeichnung,EQ-Klasse,EQ-Klasse-Bezeichnung,Gewerk,Anlagenausprägung,EQ-Menge,EQ-Merkmal_001,...,EQ-Merkmal_045,EQ-Merkmal_046,EQ-Merkmal_047,EQ-Merkmal_048,EQ-Merkmal_049,EQ-Merkmal_050,EQ-Merkmal_051,EQ-Merkmal_052,Zugeordnete Artikelnummer,Artikel Kurztext
0,21083,NaN,1000005794,Schutzraumlüftung - Mitt.(Außer Betrieb),31211,Lüftungsanlagen,RLT,Schutzraumlüft. ohne Sandfilt.,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.03.01.000.09.01,334 - Schiebetür (ohne Fluchtwegsfunktion); kr...
1,21083,NaN,1000005932,RLT Batterieraum alt - Mittelbau,31211,Lüftungsanlagen,RLT,Zu-/Abluftanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.03.01.021.00.00,431 - Zu- und Abluftanlage - Frequenzumformer ...
2,21083,NaN,1000005934,RLT Gleichr. alt - Mittelbau,31211,Lüftungsanlagen,RLT,Zu-/Abluftanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.03.01.021.00.00,431 - Zu- und Abluftanlage - Frequenzumformer ...
3,21083,NaN,1000025596,"Gebäude-Blitzschutz-, Erdungsanl., Pot.",3330,Blitzschutz-/Erdungsanlage,Elektro Blitzschutz,Blitzschutz/Erdung Gebäude,1,Ableiter Anzahl aussen: 11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446.01.01.000.00.00,446 - Innerer Blitzschutz (Potenzialausgleich)
4,21083,NaN,1000026066,Beschallungs-/Sprechanlagen,3520,Elektroakustische Anlagen,Haustechn Einricht,Beschallungsanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,454.01.01.000.00.00,454 - Elektroakustische Anlagen - Zentraleinheit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,26287,NaN,1010136947,Trinkwasserspender Eden,3710,Warenverkaufseinrichtungen,Haustechn Einricht,Getränkeschank-und-zapfanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.03.01.000.00.00,412 - Wasserverteilung (inkl. Rohrnetz und Abs...
969,26287,NaN,1010136948,Trinkwasserspender Eden,3710,Warenverkaufseinrichtungen,Haustechn Einricht,Getränkeschank-und-zapfanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.03.01.000.00.00,412 - Wasserverteilung (inkl. Rohrnetz und Abs...
970,26287,NaN,1010136949,Trinkwasserspender Eden,3710,Warenverkaufseinrichtungen,Haustechn Einricht,Getränkeschank-und-zapfanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.03.01.000.00.00,412 - Wasserverteilung (inkl. Rohrnetz und Abs...
971,26287,NaN,1010136950,Trinkwasserspender Eden,3710,Warenverkaufseinrichtungen,Haustechn Einricht,Getränkeschank-und-zapfanlage,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.03.01.000.00.00,412 - Wasserverteilung (inkl. Rohrnetz und Abs...


In [9]:
# Excel-Datei exportieren
file_path = '/gdrive/MyDrive/AIS_Management/EP_Zuordnung.xlsx'
df_EP_Zuordnung.to_excel(file_path, index=False)